# 使用豆包模型

结论：

- LLM 是可以通过 openai_like 使用的
- Embedding LlamaIndex 不支持它，LangChain中有一个奇怪的 VolcanoEngineEmbedding 但是无法使用豆包的嵌入模型，只能使用 `bge-large-zh`，而且还有权限报错

如果要使用豆包模型，只能是嵌入模型使用其他第三方的。

## 设置和使用 LLM

In [1]:
%%time
%%capture

!pip install python-dotenv

CPU times: user 3.04 ms, sys: 7.82 ms, total: 10.9 ms
Wall time: 1.81 s


In [2]:
%%time

from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("DOUBAO_API_KEY")

CPU times: user 4.52 ms, sys: 0 ns, total: 4.52 ms
Wall time: 4.25 ms


In [3]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings

model="ep-20240918183344-lvb4l"
llm = OpenAILike(
    model=model, 
    api_base="https://ark.cn-beijing.volces.com/api/v3", 
    api_key=api_key,
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

CPU times: user 2.79 s, sys: 375 ms, total: 3.17 s
Wall time: 2.78 s


In [4]:
%%time

resp = llm.stream_complete("介绍下什么叫置信区间")

for r in resp:
    print(r.delta, end="")

置信区间是在统计学中用于推断总体参数的一个重要概念。

它指的是在一定的置信水平下，总体参数的取值范围。具体来说，给定一个样本数据，通过计算统计量（如样本均值、样本比例等），并结合样本的标准差等信息，可以构造出一个区间，使得有一定的置信水平（通常为 95%或 99%等）认为总体参数落在这个区间内。

例如，对于一个样本均值，我们可以计算出它的置信区间。如果我们说样本均值的置信区间为（a，b），那么这意味着我们有一定的把握认为总体均值的真实值落在区间（a，b）内。这个区间的宽度反映了我们对总体参数估计的精确程度和不确定性程度。

置信区间的作用主要有以下几点：
1. 提供了对总体参数的一种估计范围，有助于我们对总体情况进行初步的了解和推断。
2. 可以用于假设检验，判断样本数据是否支持关于总体参数的某个假设。
3. 帮助我们在有限的样本信息基础上对总体进行一定程度的估计和推测。

在实际应用中，选择合适的置信水平和计算置信区间的方法是非常重要的，需要根据具体的研究问题和数据特点来确定。CPU times: user 156 ms, sys: 7.9 ms, total: 164 ms
Wall time: 9.38 s


## 设置和使用 Embedding

In [17]:
%%time

import os
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
    base_url="https://ark.cn-beijing.volces.com/api/v3",
)

print("----- embeddings request -----")
resp = client.embeddings.create(
    model="ep-20240918183239-l2jqq",
    input=["花椰菜又称菜花、花菜，是一种常见的蔬菜。"],
    encoding_format="float"
)
resp.data[0].embedding[:5]

----- embeddings request -----
CPU times: user 52.7 ms, sys: 10.7 ms, total: 63.4 ms
Wall time: 184 ms


[-0.345703125, -1.8671875, -1.8828125, -1.703125, -1.9609375]

In [19]:
%%time
%%capture

!pip install volcengine

CPU times: user 46.8 ms, sys: 8.78 ms, total: 55.6 ms
Wall time: 15.3 s


In [20]:
%%time

"""For basic init and call"""

# https://python.langchain.com/docs/integrations/text_embedding/volcengine/

import os

from langchain_community.embeddings import VolcanoEmbeddings

embed = VolcanoEmbeddings(
    volcano_ak="xxx", 
    volcano_sk="xxx")
print("embed_documents result:")
res = embed.embed_documents(["风急天高猿啸哀"])
res[0][:5]

embed_documents result:


ValueError: embed by volcengine Error: Detailed exception information is listed below.
req_id: 202409181911547458221C04914ECC70F6
code_n: 1709818
code: UnauthorizedUserForEndpoint
message: 没有访问该模型服务节点的权限